In [1]:
import numpy as np, pandas as pd, bs4

In [147]:
from urllib.request import urlopen as uReq
from bs4 import BeautifulSoup as soup
import requests

In [3]:
myurl = 'https://web-as.tamu.edu/gradereports/'
registrar_page = uReq(myurl)
page_html = registrar_page.read()
registrar_page.close()
page_soup = soup(page_html, 'html.parser')
diff_distributions = page_soup.findAll('table', {})

7

In [124]:
index = []
years = []
colleges = []
sems = []
for i in range(3):
    index.append([])
    for row in diff_distributions[i+1].findAll('option'):
        index[i].append(row['value'])
    years.append(index[i][:index[i].index('A')])
    colleges.append(index[i][index[i].index('C')+1:])
    sems.append(index[i][index[i].index('A'):index[i].index('C') +1])


In [185]:
def get_df(year, sem, college): 
    url = "https://web-as.tamu.edu/GradeReports/PDFReports/" + str(year) + str(sem) + "/grd" + str(year) + str(sem) + str(college) + ".pdf"
    print(url)
    try:
        response = requests.get(url)
        response.raise_for_status()
    except requests.HTTPError:
        return [[],[]]
    df = tabula.read_pdf(url,pages = 'all')
    cols = ['Year-sem','SECTION', "A",'B', "C", 'D', 'F','TOTAL A-F','GPA','I','S','U','Q','X','TOTAL','INSTRUCTOR' ]    
    combined_df = pd.DataFrame(columns = cols)
    for i in range(len(df)):
        try:
            df[i].loc[-1] = df[i].columns.values
            df[i].index = df[i].index + 1  # shifting index
            df[i].sort_index(inplace=True)
            df[i].insert(0, 'Year-sem', [str(year) + str(sem) for i in range(df[i].shape[0])])
            df[i].columns = cols            
            
            combined_df = combined_df.append(df[i], ignore_index = True)
        except ValueError:
            continue
    combined_df = combined_df.iloc[::2]
    course_df = combined_df[combined_df['SECTION'] == "COURSE TOTAL:"]
    section_df = combined_df[combined_df['SECTION'] != "COURSE TOTAL:"].iloc[:-2]
    return course_df, section_df

In [232]:
cols = ['Year-Sem','SECTION', "A", 'B', "C", 'D', 'F','TOTAL A-F','GPA','I','S','U','Q','X','TOTAL','INSTRUCTOR' ]


In [259]:
merged_course_df = pd.DataFrame(columns = cols)
merged_section_df = pd.DataFrame(columns = cols)
for year in [2021]:
    print(year)
    for college in colleges[0]:
        if "PROF" in college:
            continue
        for sem_val in range(3):
            merged_course_df = merged_course_df.append(get_df(year, sem_val+1, college)[0])
            merged_section_df = merged_section_df.append(get_df(year, sem_val+1, college)[1], ignore_index = True) 

2021
https://web-as.tamu.edu/GradeReports/PDFReports/20211/grd20211AE.pdf
https://web-as.tamu.edu/GradeReports/PDFReports/20211/grd20211AE.pdf
https://web-as.tamu.edu/GradeReports/PDFReports/20212/grd20212AE.pdf
https://web-as.tamu.edu/GradeReports/PDFReports/20212/grd20212AE.pdf
https://web-as.tamu.edu/GradeReports/PDFReports/20213/grd20213AE.pdf
https://web-as.tamu.edu/GradeReports/PDFReports/20213/grd20213AE.pdf
https://web-as.tamu.edu/GradeReports/PDFReports/20211/grd20211AG.pdf
https://web-as.tamu.edu/GradeReports/PDFReports/20211/grd20211AG.pdf
https://web-as.tamu.edu/GradeReports/PDFReports/20212/grd20212AG.pdf
https://web-as.tamu.edu/GradeReports/PDFReports/20212/grd20212AG.pdf
https://web-as.tamu.edu/GradeReports/PDFReports/20213/grd20213AG.pdf
https://web-as.tamu.edu/GradeReports/PDFReports/20213/grd20213AG.pdf
https://web-as.tamu.edu/GradeReports/PDFReports/20211/grd20211AR.pdf
https://web-as.tamu.edu/GradeReports/PDFReports/20211/grd20211AR.pdf
https://web-as.tamu.edu/Grade

ConnectionError: HTTPSConnectionPool(host='web-as.tamu.edu', port=443): Max retries exceeded with url: /GradeReports/PDFReports/20211/grd20211GE.pdf (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x000002762FF45B48>: Failed to establish a new connection: [WinError 10060] A connection attempt failed because the connected party did not properly respond after a period of time, or established connection failed because connected host has failed to respond'))

In [262]:
merged_section_df

,Year-Sem,SECTION,A,B,C,D,F,TOTAL A-F,GPA,I,S,U,Q,X,TOTAL,INSTRUCTOR,Year-sem
0,NaN,ASCC-101-502,7,7.1,2,1,2.1,19,2.842,0,0.1,0.2,0.3,0.4,19.1,MCCLURE M,20211
1,NaN,ASCC-101-503,5,12,6,0,2,25,2.72,0,0,0,0,0,25,JONES M,20211
2,NaN,ASCC-289-200,12,7,9,0,1,29,0.000,0.1,0.2,0.3,0.4,0.5,29.1,MCCLURE M,20212
3,NaN,ASCC-289-201,25,3,0,1,0,29,0.000,0,0,0,0,0,29,MCCLURE M,20212
4,NaN,ASCC-289-202,26,4,0,0,0,30,0.000,0,0,0,0,0,30,MCCLURE M,20212
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5512,NaN,MARA-664-699,3,2,0,0,0,5,3.6,0,0,0,0,0,5,WANG P,20212
5513,NaN,MGMT-481-100,3,5,0,0,0,8,3.375,0,0,0,0,0,8,KNOX K,20212
5514,NaN,SCMT-303-199,2,3,3,1,1,10,2.4,0,0,0,0,0,10,HUFTON A,20212
5515,NaN,SCMT-364-199,9,3,6,1,1,20,2.9,0,0,0,1,0,21,WANG P,20212


In [82]:
"""
url = "https://web-as.tamu.edu/GradeReports/PDFReports/20211/grd20211DN_PROF.pdf"
import tabula
df = tabula.read_pdf(url,pages = 'all')
"""

In [ ]:
"""
cols = ['SECTION', "A", "B+",'B', "C+", "C", 'D', 'F','TOTAL A-F','GPA','I','S','U','X','TOTAL','INSTRUCTOR' ]
combined_df = pd.DataFrame(columns = cols)
for i in range(len(df)-1):
    df[i].loc[-1] = df[i].columns.values
    df[i].index = df[i].index + 1  # shifting index
    df[i].sort_index(inplace=True)
    df[i].columns = cols
    combined_df = combined_df.append(df[i], ignore_index = True)
combined_df = combined_df.iloc[::2]
combined_df.head()
"""

In [ ]:
merged_course_df.to_csv('merged_course.csv')
merged_section_df.to_csv('merged_section.csv')

In [ ]:
merged_course_df = pd.read_csv('merged_course.csv')
merged_section_df = pd.read_csv('merged_section.csv')

In [263]:
merged_section_df = merged_section_df.drop(columns = ['Year-Sem'])
merged_course_df = merged_course_df.drop(columns=['Year-Sem'])
merged_section_df.head()

,SECTION,A,B,C,D,F,TOTAL A-F,GPA,I,S,U,Q,X,TOTAL,INSTRUCTOR,Year-sem
0,ASCC-101-502,7,7.1,2,1,2.1,19,2.842,0,0.1,0.2,0.3,0.4,19.1,MCCLURE M,20211
1,ASCC-101-503,5,12,6,0,2,25,2.72,0,0,0,0,0,25,JONES M,20211
2,ASCC-289-200,12,7,9,0,1,29,0.000,0.1,0.2,0.3,0.4,0.5,29.1,MCCLURE M,20212
3,ASCC-289-201,25,3,0,1,0,29,0.000,0,0,0,0,0,29,MCCLURE M,20212
4,ASCC-289-202,26,4,0,0,0,30,0.000,0,0,0,0,0,30,MCCLURE M,20212


In [264]:
import kmapper as km
import numpy as np
from sklearn import cluster
from sklearn.preprocessing import MinMaxScaler
from kmapper import jupyter
import matplotlib.pyplot as plt
import sklearn


In [333]:
num_col = [ "A", 'B', "C", 'D', 'F','TOTAL A-F','GPA','I','S','X','TOTAL','Year-sem' ]

In [334]:
merged_section_df_num = merged_section_df[num_col].astype(int, errors = 'ignore')
merged_section_df_num[merged_section_df_num['S'].apply(lambda x: isinstance(x, float))]
merged_section_df_num = merged_section_df_num.drop(np.where(merged_section_df_num['S'] == 'Unnamed: 0')[0])

In [335]:
merged_section_df_num

,A,B,C,D,F,TOTAL A-F,GPA,I,S,X,TOTAL,Year-sem
0,7,7.1,2,1,2.1,19,2.842,0,0.1,0.4,19.1,20211
1,5,12,6,0,2,25,2.72,0,0,0,25,20211
2,12,7,9,0,1,29,0.000,0.1,0.2,0.5,29.1,20212
3,25,3,0,1,0,29,0.000,0,0,0,29,20212
4,26,4,0,0,0,30,0.000,0,0,0,30,20212
...,...,...,...,...,...,...,...,...,...,...,...,...
5512,3,2,0,0,0,5,3.6,0,0,0,5,20212
5513,3,5,0,0,0,8,3.375,0,0,0,8,20212
5514,2,3,3,1,1,10,2.4,0,0,0,10,20212
5515,9,3,6,1,1,20,2.9,0,0,0,21,20212


In [336]:

mapper = km.KeplerMapper(verbose=1)
lens = mapper.fit_transform(merged_section_df_num, projection = sklearn.manifold.TSNE())
#default scalar is MinMaxScalar() and default projection is sum default distance_matrix = False
#eps in DBSCAN represents the maximum distance between 2 samples to be considered as neighbors.
#min_samples number of samples/points in the neighborhood to be considered a core point.
graph = mapper.map(lens,
                   clusterer=sklearn.cluster.DBSCAN(eps=0.2, min_samples=1, metric = 'cosine'),
                   cover=km.Cover(n_cubes=25, perc_overlap=0.8), remove_duplicate_nodes=True)
#perc_overlap is amount of overlap between adjacent cubes.
#this graph is a simplicial complex
mapper.visualize(graph,
                 path_html="new.html")
km.draw_matplotlib(graph)
plt.show()
jupyter.display("new.html")

KeplerMapper(verbose=1)
..Composing projection pipeline of length 1:
	Projections: TSNE(angle=0.5, early_exaggeration=12.0, init='random', learning_rate=200.0,
     method='barnes_hut', metric='euclidean', min_grad_norm=1e-07,
     n_components=2, n_iter=1000, n_iter_without_progress=300, n_jobs=None,
     perplexity=30.0, random_state=None, verbose=0)
	Distance matrices: False
	Scalers: MinMaxScaler(copy=True, feature_range=(0, 1))
..Projecting on data shaped (5515, 12)

..Projecting data using: 
	TSNE(angle=0.5, early_exaggeration=12.0, init='random', learning_rate=200.0,
     method='barnes_hut', metric='euclidean', min_grad_norm=1e-07,
     n_components=2, n_iter=1000, n_iter_without_progress=300, n_jobs=None,
     perplexity=30.0, random_state=None, verbose=1)


..Scaling with: MinMaxScaler(copy=True, feature_range=(0, 1))

Mapping on data shaped (5515, 12) using lens shaped (5515, 12)



MemoryError: 

In [242]:
import pandas as pd, plotly.express as px
import plotly.graph_objects as go
fig = px.scatter(merged_section_df.dropna(), x = 'INSTRUCTOR', y= 'GPA',
           size = 'A' ,size_max = 20,  color = 'TOTAL',facet_col_wrap = 10, height = 1200)
fig.update_layout(
    title={'text' : 'Instructor Grading','x':0.5, 'xanchor': 'center'}
)
fig.show()

TypeError: unsupported operand type(s) for /: 'str' and 'int'

In [226]:
cols

['Year-Sem',
 'SECTION',
 'A',
 'B',
 'C',
 'D',
 'F',
 'TOTAL A-F',
 'GPA',
 'I',
 'S',
 'U',
 'Q',
 'X']